In [1]:
from sage.all import *
from sage.symbolic.operators import add_vararg, mul_vararg

$\tau$ denotes the Tits endomorphism

In [2]:
tau = sage.symbolic.function_factory.function('tau', nargs=1, print_latex_func=lambda _, arg: '{}^\\tau'.format(arg))

In [3]:
a, b = var('a, b')
q = var('q')
u, v = var('u, v')
x, y, z, t = var('x, y, z, t')
variables = [a, b, q, u, v, x, y, z, t]

The utility functions defined below allow to manipulate the expressions involving $\tau$, reduction of expressions mod 2 and mapping those over matrices.

In [4]:
# taken with changes from https://wiki.sagemath.org/symbolics/rewrite

def mapexpression(expr, fct, level, addDepth=0, mulDepth=0):
     def mapex(expr, depth):               # a very local function
        if expr.is_integer():
            return expr
        opor = expr.operator()
        opands = expr.operands()
        if (opor is None):
            return expr       # a leaf in the expression tree
        if (opor == operator.add):           # recursive call thru sum
            opands = map(lambda ex: mapex(ex, depth + addDepth), opands)
            return sum(opands)
        if (opor == operator.mul):           # recursive call thru mul
            opands = map(lambda ex: mapex(ex, depth + mulDepth), opands)
            return prod(opands)
        if (level == -1) or (level[-1] >= depth):  # recursive call over operands
            opands = map(lambda ex: mapex(ex, depth + 1), opands)
        if level == -1 or depth in level:  # root of the subtree must be changed
            return fct(opor, opands)
        return opor(*opands)  # opands may or may not be changed by a recursive call
     return mapex(expr, 0)


# a way to circumvent a conflict between built-in `pow` functions
sagepow = (1/x).operator()
    

def tits_endomorphism_rules(opor, opands):
    """
    tau(a ± b) => tau(a) ± tau(b)
    tau(a * b^n) => tau(a) * tau(b)^n
    tau(1) => 1
    """
    opands = list(opands)
    if opor == tau:
        opand = opands[0]
        opand_opor = opand.operator()
        if opand_opor is not None:
            opand_opands = opand.operands()
            if opand_opor == add_vararg or opand_opor == mul_vararg:
                return opand_opor(*[tau(o) for o in opand_opands])
            elif opand_opor == sagepow:
                return pow(tau(opand_opands[0]), opand_opands[1])
            elif opand_opor == tau:
                return opand_opands[0]**2
        if opand.is_integer():
            return opand
    return opor(*opands)


def rewrite(expr):
    stable = false
    while not stable:
        new_expr = mapexpression(expr, tits_endomorphism_rules, -1)
        stable = (new_expr == expr)
        expr = new_expr
    return expr


def map_rewrite(m):
    return m.apply_map(rewrite)


def map_simplify_rational(m):
    return m.apply_map(lambda e: e.simplify_rational(algorithm='simple'))


def reduce_mod2(expr):
    tempvars = list(var(['tau_{}'.format(v) for v in variables]))
    numer_denom = []
    for e in expr.numerator_denominator():
        e = e.subs({tau(v): tau_v for v, tau_v in zip(variables, tempvars)})
        Rmod2 = PolynomialRing(GF(2), variables + tempvars)
        e = Rmod2(e)
        e = e.change_ring(ZZ)
        e = e.subs(**{str(tau_v): tau(v) for tau_v, v in zip(tempvars, variables)})
        numer_denom.append(e)
    return numer_denom[0] / numer_denom[1]


def map_reduce_mod2(m):
    return m.apply_map(lambda e: reduce_mod2(e))

Define the basic constructions inside Suzuki group: the non-trivial torus normalizer element $w$, the positive and negative root elements $x_\pm(a,b)$ and the torus elements $h(\varepsilon)$.

In [5]:
w = matrix([[0, 0, 0, 1], [0, 0, 1, 0], [0, 1, 0, 0], [1, 0, 0, 0]])
w.set_immutable()


def xp(a, b):
    mat = matrix(
        [
            [1, a, b + a * tau(a), a * b + tau(b) + a**2 * tau(a)],
            [0, 1, tau(a), b],
            [0, 0, 1, a],
            [0, 0, 0, 1]
        ]
    )
    # apply rewriting to each entry to avoid entries like `0^\tau` for specific parameters
    return map_rewrite(mat)


def xm(a, b):
    return w * xp(a, b) * w


def h_diag(eps):
    mat = diagonal_matrix([eps, tau(eps)/eps, eps/tau(eps), 1/eps])
    return map_rewrite(mat)

In [6]:
show(xp(a, b), xm(a,b), xp(0, x), xm(0, y))

[                        1                         a              a*tau(a) + b a^2*tau(a) + a*b + tau(b)]
[                        0                         1                    tau(a)                         b]
[                        0                         0                         1                         a]
[                        0                         0                         0                         1] [                        1                         0                         0                         0]
[                        a                         1                         0                         0]
[                        b                    tau(a)                         1                         0]
[a^2*tau(a) + a*b + tau(b)              a*tau(a) + b                         a                         1] [     1      0      x tau(x)]
[     0      1      0      x]
[     0      0      1      0]
[     0      0      0      1] [     1      0      0      0]
[     0      1      0      0]
[     y      0      1      0]
[tau(y)      y      0      1]

We now introduce the two products $x_-(0,u) x_+(a,b)$ and $x_+(0,x)x_-(0,y)x_+(0,z)x_-(0,t)$, which will coincide, provided suitable values for the variables $u,x,y,z,t$ and under the assumption $b \neq 0$.

In [7]:
g = xm(0, u) * xp(a, b)
show(g)

[                                           1                                            a                                 a*tau(a) + b                    a^2*tau(a) + a*b + tau(b)]
[                                           0                                            1                                       tau(a)                                            b]
[                                           u                                          a*u                         (a*tau(a) + b)*u + 1            (a^2*tau(a) + a*b + tau(b))*u + a]
[                                      tau(u)                                 a*tau(u) + u             u*tau(a) + (a*tau(a) + b)*tau(u) b*u + (a^2*tau(a) + a*b + tau(b))*tau(u) + 1]

In [8]:
pqpq = xp(0, x) * xm(0, y) * xp(0, z) * xm(0, t)
show(pqpq)

[((x*y + tau(x)*tau(y) + 1)*z + x)*t + x*y + (y*z*tau(x) + (x*y + tau(x)*tau(y) + 1)*tau(z) + tau(x))*tau(t) + tau(x)*tau(y) + 1                                                           (y*z*tau(x) + (x*y + tau(x)*tau(y) + 1)*tau(z) + tau(x))*t + y*tau(x)                                                                                                 (x*y + tau(x)*tau(y) + 1)*z + x                                                                          y*z*tau(x) + (x*y + tau(x)*tau(y) + 1)*tau(z) + tau(x)]
[                                                           t*x*z*tau(y) + (x*tau(y)*tau(z) + (x*y + 1)*z + x)*tau(t) + x*tau(y)                                                                                 (x*tau(y)*tau(z) + (x*y + 1)*z + x)*t + x*y + 1                                                                                                                      x*z*tau(y)                                                                                               x*tau(y)*tau(z) + (x*y + 1)*z + x]
[                                                                                              y*tau(t)*tau(z) + (y*z + 1)*t + y                                                                                                                      t*y*tau(z)                                                                                                                         y*z + 1                                                                                                                        y*tau(z)]
[                                                                         t*z*tau(y) + (y*z + tau(y)*tau(z) + 1)*tau(t) + tau(y)                                                                                                 (y*z + tau(y)*tau(z) + 1)*t + y                                                                                                                        z*tau(y)                                                                                                         y*z + tau(y)*tau(z) + 1]

In [9]:
values = {
    t: u * a / (u * g[0, 3] + a),
    y: u * g[0, 2] / z,
    z: (g[0, 3] * tau(u) + u * b + 1) * x + b,
    x: tau(a) / (tau(u) * g[0,2] + u * tau(a)),
    u: a**2 * tau(a) / (b * tau(b)),
}

for vrb, vrb_sub in values.items():
    show(vrb, ' = ', vrb_sub)

t ' = ' a*u/((a^2*tau(a) + a*b + tau(b))*u + a)

y ' = ' (a*tau(a) + b)*u/z

z ' = ' (b*u + (a^2*tau(a) + a*b + tau(b))*tau(u) + 1)*x + b

x ' = ' tau(a)/(u*tau(a) + (a*tau(a) + b)*tau(u))

u ' = ' a^2*tau(a)/(b*tau(b))

The values for the variables can be expanded in terms of $a$ and $b$, giving large and complicated expressions, suitable for machine verification.

In [10]:
values_expanded = copy(values)

vrb_list = list(values)

for i, vrb in reversed(list(enumerate(vrb_list))):
    vrb_sub = values_expanded[vrb]
    for c in vrb_list[:i]:
        c_sub = values_expanded[c]
        c_sub_expanded = c_sub({vrb: vrb_sub})
        c_sub_expanded = c_sub_expanded.simplify_rational()
        c_sub_expanded = rewrite(c_sub_expanded)
        values_expanded[c] = c_sub_expanded

for vrb in vrb_list:
    vrb_sub = values_expanded[vrb]
    vrb_sub = reduce_mod2(vrb_sub)
    print(vrb, '=', vrb_sub)
    values_expanded[vrb] = vrb_sub

t = a^2*tau(a)/(a^3*tau(a)^2 + a^2*b*tau(a) + a*tau(a)*tau(b) + b*tau(b))
y = (a^6*tau(a)^4 + a^5*b*tau(a)^3)/(a^4*b*tau(a)^3*tau(b) + a^2*b^3*tau(a)*tau(b) + a^2*b*tau(a)^2*tau(b)^2 + b^3*tau(b)^2)
z = (a^4*tau(a)^3 + a^2*b^2*tau(a) + a^2*tau(a)^2*tau(b) + b^2*tau(b))/(a^3*tau(a)^2)
x = b^2*tau(b)/(a^3*tau(a)^2)
u = a^2*tau(a)/(b*tau(b))


In [11]:
g_eval = map_rewrite(g.subs(values_expanded))

In [12]:
pqpq_factors = [
    xp(0, values_expanded[x]),
    xm(0, values_expanded[y]),
    xp(0, values_expanded[z]),
    xm(0, values_expanded[t])
]
pqpq_eval = prod(pqpq_factors)

The difference between these two products, once reduced mod 2, becomes 0, proving the decomposition.

In [13]:
map_reduce_mod2(pqpq_eval - g_eval)

[0 0 0 0]
[0 0 0 0]
[0 0 0 0]
[0 0 0 0]